In [4]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as sci
import time 
import threading
import sys

## Parameters

In [5]:
class Data:
    def __init__(self):
        "Dimension gives the dimension of the system"
        # ------- TO DO ---------
        # Depending on the system this has to be adjusted
        self.dimension = 12
        
        "Alpha describes the rate of infection"
        self.alpha = 0 # Dimensions day^(-1)

        "Beta describes the recovery rate"
        self.beta = 1/14 # Dimensions day^(-1)

        "p is the probability to die from the disease"
        self.p = 2.64e-2
        
        """
        Commuters is an array describing the commuters from and to the different cells. 
        The structure is: The entries in column i are how many commute from i to the cell of the row.
                          Thus, the entries of the row i describe how many commute from different cells to cell i.
        """
        # ------- TO DO ---------
        # fill the matrix
        self.commuters = np.array([[], ... , []])
        
        
        
    def commutersFrom(self, cfrom):
        """
        Function to return an array with number of commuters from cfrom.
        
        Input: int cfrom; from what cell to the commuters come from.
        
        Output: array; array containing the commuters from cfrom.
        """
        # ------- TO DO ---------
        # optimize the algorithm
        "Sorting out the necessary entries into array a."
        a = np.zeros(self.dimension)
        for i in range(self.dimension):
            a[i] = self.commuters[i][cfrom]
            
        return a 
    
    def commutersTo(self, cto):
        """
        Function to return an array with number of commuters coming to cto from other cells.
        
        Input: int cto; to what cell do the commuters travel to.
        
        Output: array; array containing the commuters to cto.
        """
        return self.commuters[cto][:]
       
        
        

0.0264


## Functions